<a href="https://colab.research.google.com/github/AmeerHamza0220/Deep-Learning/blob/main/CNNBasic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
(x_train, y_train), (x_test, y_test)=tf.keras.datasets.cifar10.load_data()
num_classes = 10
img_rows, img_cols = 32,32
num_channels = 3

In [ ]:
x_train.shape

In [ ]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512,activation='tanh'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(128,activation='tanh'))


model.add(tf.keras.layers.Dense(num_classes,activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
callbacks = [tf.keras.callbacks.TensorBoard('./keras')]
model.fit(x_train, y_train, epochs=25, verbose=1, validation_data=(x_test, y_test), callbacks=callbacks)

Basic stuff using convulution and pooling

In [ ]:
!wget https://upload.wikimedia.org/wikipedia/commons/d/da/Epimachus_meyeri_-Papua_New_Guinea_-male-8.jpg

In [ ]:
import matplotlib
from matplotlib import pyplot as plt
from skimage import io        # Package to simply read images

In [ ]:
from PIL import Image
image = Image.open('/content/Epimachus_meyeri_-Papua_New_Guinea_-male-8.jpg').convert('L').resize((600,600))


In [ ]:
image.shape

In [ ]:
plt.imshow(image, cmap=plt.cm.gray)

In [ ]:
image = tf.keras.preprocessing.image.img_to_array(image)

In [ ]:
image = tf.expand_dims(image, axis=0)
 #tensorflow work on batches 

In [ ]:

print("Tensor shape: {}".format(image.shape))

 we define a 3x3 filter (or kernel) commonly used to blur images (Gaussian blur):

In [ ]:
kernel = tf.constant([[1 / 16, 2 / 16, 1 / 16],
                      [2 / 16, 4 / 16, 2 / 16],
                      [1 / 16, 2 / 16, 1 / 16]], tf.float32, name="gaussian_kernel")

In [ ]:
kernel = tf.expand_dims(tf.expand_dims(kernel, axis=-1), axis=-1)
kernel.shape

In [ ]:
blurred_image = tf.nn.conv2d(image, kernel, strides=[1, 1, 1, 1], padding="SAME")

In [ ]:
blurred_res = blurred_image.numpy()
# We "unbatch" our result by selecting the first (and only) image; we also remove the depth dimension:
blurred_res = blurred_res[0, ..., 0]

plt.imshow(blurred_res, cmap=plt.cm.gray)

a kernel for contour detection is used. This kernel is defined as follows:

In [ ]:
kernel = tf.constant([[-1, -1, -1],
                      [-1,  8, -1],
                      [-1, -1, -1]], tf.float32, name="edge_kernel")
kernel = tf.expand_dims(tf.expand_dims(kernel, axis=-1), axis=-1)

In [ ]:
edge_image = tf.nn.conv2d(image, kernel, strides=[1, 2, 2, 1], padding="SAME")
edge_res = edge_image.numpy()[0, ..., 0]
plt.imshow(edge_res, cmap=plt.cm.gray)

For max-pooling and average-pooling, the values in each window are aggregated into a single output, applying respectively the max or averaging operation. Once again, we use the low-level TensorFlow API to reproduce the results shown in the chapter:

In [ ]:
avg_pooled_image = tf.nn.avg_pool(image, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
avg_res = avg_pooled_image.numpy()[0, ..., 0]
plt.imshow(avg_res, cmap=plt.cm.gray)

In [ ]:
max_pooled_image = tf.nn.max_pool(image, ksize=[1, 10, 10, 1], strides=[1, 2, 2, 1], padding="SAME")
max_res = max_pooled_image.numpy()[0, ..., 0]
plt.imshow(max_res, cmap=plt.cm.gray)

Simple convulutional network

In [ ]:
class LeNet(tf.keras.Model):
    def __init__(self,num_classes):
        super(LeNet, self).__init__()
        self.conv1=tf.keras.layers.Conv2D(20,kernel_size=5,padding="same",activation='relu')
        self.conv2=tf.keras.layers.Conv2D(30,kernel_size=5,activation='relu')
        self.maxpool=tf.keras.layers.MaxPooling2D(pool_size=2)
        self.flatten=tf.keras.layers.Flatten()
        self.dense1=tf.keras.layers.Dense(400,activation='relu')
        self.dense2=tf.keras.layers.Dense(num_classes,activation='relu')
    def call(self,input):
        x=self.conv1(input)
        x=self.maxpool(x)
        x=self.conv2(x)
        x=self.maxpool(x)
        x=self.flatten(x)
        x=self.dense1(x)
        x=self.dense2(x)
        return x

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
class LeNet5(Model):
    
    def __init__(self, num_classes):
        """
        Initialize the model.
        :param num_classes:     Number of classes to predict from
        """
        super(LeNet5, self).__init__()
        # We instantiate the various layers composing LeNet-5:
        # self.conv1 = SimpleConvolutionLayer(6, kernel_size=(5, 5))
        # self.conv2 = SimpleConvolutionLayer(16, kernel_size=(5, 5))
        # ... or using the existing and (recommended) Conv2D class:
        self.conv1 = Conv2D(6, 5, padding='same', activation='relu')
        self.conv2 = Conv2D(16, 5, activation='relu')
        self.max_pool = MaxPooling2D(pool_size=(2, 2))
        self.flatten = Flatten()
        self.dense1 = Dense(120, activation='relu')
        self.dense2 = Dense(84, activation='relu')
        self.dense3 = Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        """
        Call the layers and perform their operations on the input tensors
        :param inputs:  Input tensor
        :return:        Output tensor
        """
        x = self.max_pool(self.conv1(inputs))        # 1st block
        x = self.max_pool(self.conv2(x))             # 2nd block
        x = self.flatten(x)
        x = self.dense3(self.dense2(self.dense1(x))) # dense layers
        return x

In [ ]:
input_shape=32,32,3


model = LeNet5(10)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=[tf.keras.metrics.sparse_categorical_accuracy])
callbacks = [
    # Callback to interrupt the training if the validation loss (`val_loss`) stops improving for over 3 epochs:
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
    # Callback to log the graph, losses and metrics into TensorBoard (saving log files in `./logs` directory):
    tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True)]
# batched_input_shape = tf.TensorShape((None, *input_shape))
# model.build(input_shape=batched_input_shape)
# model.summary()

In [ ]:
x_train=tf.cast(x_train, tf.float32)
x_test=tf.cast(x_test, tf.float32)

history = model.fit(x_train, y_train,
                    batch_size=32, epochs=10, validation_data=(x_test, y_test), 
                    verbose=1,  # change to `verbose=1` to get a progress bar
                                # (we opt for `verbose=2` here to reduce the log size)
                    callbacks=callbacks)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs